In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.preprocessing import MinMaxScaler
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix
scaler = MinMaxScaler()

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\hp\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
Survival = pd.read_csv('Titanic train_data.csv')
Survival.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
Survival1 = Survival.drop(['PassengerId','Name','Ticket','Cabin','Embarked'], axis = 1)
Survival1.replace('female',0,inplace = True)
Survival1.replace('male',1, inplace = True)
Survival2 = Survival1.fillna(0)
Survival2.head()
Survival2.shape

(891, 7)

In [5]:
X = Survival2[['Age','Pclass','Sex','Parch','SibSp','Fare']]
y = Survival2['Survived']
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
survival = svm.SVC(C=2,gamma =2,random_state = 67)
survival.fit(X_train_scaled,y_train)

SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='rbf',
  max_iter=-1, probability=False, random_state=67, shrinking=True,
  tol=0.001, verbose=False)

In [14]:
y_pred = survival.predict(X_test)
Train_accuracy = survival.score(X_train_scaled,y_train)
Test_accuracy = survival.score(X_test_scaled,y_test)
print("{0:.2f}".format(Train_accuracy*100))
print("{0:.2f}".format(Test_accuracy*100))

83.83
80.27


In [8]:
Cs = np.array([11,12,13,14,15,16,17,18,19,10,20])
gammas = np.array([11,12,13,14,15,16,17,18,19,10,20])
model = svm.SVC()
grid = GridSearchCV(estimator=model, param_grid=dict(gamma=gammas,C = Cs),cv=4)
grid.fit(X_train_scaled,y_train)
print(grid)
print(grid.best_score_)
print(grid.best_estimator_.gamma)
print(grid.best_estimator_.C)

GridSearchCV(cv=4, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'gamma': array([11, 12, 13, 14, 15, 16, 17, 18, 19, 10, 20]), 'C': array([11, 12, 13, 14, 15, 16, 17, 18, 19, 10, 20])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)
0.8098802395209581
11
13


In [9]:
cnf_matrix = confusion_matrix(y_test,y_pred)
cnf_matrix

array([[139,   0],
       [ 84,   0]], dtype=int64)

In [12]:
parameters = {
    'gamma': [1,2,3,4,5,6],
    'C': [2,4,6,8,10]}

selector = GridSearchCV(SVC(), parameters, scoring='accuracy') 
selector.fit(X_train_scaled, y_train)

print('Best parameter set found:')
print(selector.best_params_)
print('Detailed grid scores:')
means = selector.cv_results_['mean_test_score']
stds = selector.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, selector.cv_results_['params']):
    print('%0.3f (+/-%0.03f) for %r' % (mean, std * 2, params))
    print()


Best parameter set found:
{'C': 2, 'gamma': 2}
Detailed grid scores:
